In [2]:
# (Opcional en local/Colab) Instalar dependencias
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 3) Cargar la clave y crear el cliente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


In [4]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías avanzadas que simulan la inteligencia humana para personalizar el aprendizaje, automatizar tareas administrativas y proporcionar retroalimentación en tiempo real. Esto permite a los educadores adaptar sus métodos de enseñanza a las necesidades individuales de los estudiantes, mejorando así su experiencia y resultados académicos.


In [5]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.3,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo se entrena utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida correcta para nuevas entradas. Este enfoque se utiliza comúnmente en tareas como clasificación y regresión."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo se entrena utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida correcta para nuevas entradas. Este enfoque se utiliza comúnmente en tareas como clasificación y regresión.'}


In [6]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza un gráfico en forma de árbol para tomar decisiones basadas en preguntas sobre las características de los datos. Cada nodo interno representa una prueba en una característica, cada rama representa el resultado de esa prueba y cada hoja representa una clase o decisión final, permitiendo así clasificar o predecir resultados de manera estructurada."
}


##Ejercicios propuestos

In [7]:
for temp in [0.1, 0.5, 0.9]:
    respuesta = ask_model(
        f"Describe en una frase el impacto de la inteligencia artificial en el arte digital.",
        temperature=temp
    )
    print(f"\n🌡️ Temperature = {temp}\n{respuesta}")


🌡️ Temperature = 0.1
La inteligencia artificial ha revolucionado el arte digital al permitir la creación de obras innovadoras y personalizadas, ampliando las posibilidades creativas y desafiando las nociones tradicionales de autoría y expresión artística.

🌡️ Temperature = 0.5
La inteligencia artificial ha transformado el arte digital al permitir la creación de obras innovadoras y personalizadas, ampliando las posibilidades creativas y democratizando el acceso a herramientas artísticas avanzadas.

🌡️ Temperature = 0.9
La inteligencia artificial ha revolucionado el arte digital al permitir la generación automática de obras, la personalización a gran escala y la exploración de nuevas formas creativas, desdibujando las fronteras entre el artista y la máquina.


In [8]:
system_rule = (
    "Responde siempre en formato JSON con las claves 'tema' y 'descripcion'. ""No incluyas texto adicional fuera del JSON.")

prompt = "Explica qué es la ciberseguridad en educación."
respuesta = ask_model(
    prompt,
    system=system_rule,
    temperature=0.3)
print(respuesta)

{
  "tema": "Ciberseguridad en Educación",
  "descripcion": "La ciberseguridad en educación se refiere a las prácticas y medidas implementadas para proteger la información y los sistemas digitales utilizados en entornos educativos. Esto incluye la protección de datos personales de estudiantes y personal, la seguridad de las plataformas de aprendizaje en línea, y la prevención de ataques cibernéticos. La ciberseguridad en este contexto es crucial para garantizar un entorno de aprendizaje seguro y confiable, donde se fomente el uso responsable de la tecnología y se minimicen los riesgos asociados con el acceso a la información digital."
}


In [9]:
system = "Responde en JSON con las claves: operation, input, steps (lista), output."
prompt = (
    "Explica cómo convertir una lista de nombres en mayúsculas usando Python.")

respuesta = ask_model(
    prompt,
    system=system,
    temperature=0.4)
print(respuesta)

```json
{
  "operation": "convert_list_to_uppercase",
  "input": ["juan", "maria", "pedro", "luisa"],
  "steps": [
    "Definir una lista de nombres en minúsculas.",
    "Utilizar una comprensión de lista para iterar sobre cada nombre en la lista original.",
    "Aplicar el método .upper() a cada nombre dentro de la comprensión de lista.",
    "Almacenar el resultado en una nueva lista."
  ],
  "output": ["JUAN", "MARIA", "PEDRO", "LUISA"]
}
```


In [10]:
respuesta = ask_model(
    "Redacta una breve historia sobre un robot que aprende a pintar cuadros.",
    temperature=0.7,
    max_tokens=30)
print(respuesta)

En un futuro no muy lejano, en una pequeña ciudad donde la tecnología y el arte coexistían, vivía un robot llamado Artie. Dise
